# TP_HMM_VITERBI_ALKASSOUM_YACINE

# NB

# Lib and dataset imports

In [41]:
import math
with open("./train.ester1.cut.bio") as file:
    content = file.readlines()
content = [ elem.strip().split(" ") for elem in content]

# List of sentence
file = open("train.ester1.cut.bio", "r")
train = file.read().split("\n\n")
file.close()

# Cleansing and Stats

In [51]:
# List of tuples/dictionarys
wordDiff = {}
taggersDiff = {}
aTagger = []
aMot = []


#We split lines into tpl of words: TOKENISATION
def splitSentence( sentences ):
    sent = []

    for sentence in sentences:
        temp = []
        for tpl in sentence.split("\n"):
            temp.append(tpl.split(" "))
        sent.append(temp)
        
    return sent

for elem in content:
    if elem[0] != '':    
        wordDiff[elem[0]] = 1 if elem[0] not in wordDiff else wordDiff[elem[0]]  
        taggersDiff[elem[1]] = 1 if elem[1] not in taggersDiff else taggersDiff[elem[1]]
            

for etiq in taggersDiff:
    aTagger.append(etiq)
    
for word in wordDiff:
    aMot.append(word)
print("Nombre de mots defférents: " + str(len(wordDiff)))
print("Nombre d etiquettes différents: " + str(len(taggersDiff)))

Nombre de mots defférents: 33085
Nombre d etiquettes différents: 15


# TRIVIAL SYSTEM

In [52]:
#We split lines into tpl of words: TOKENISATION, each word with his tag
spl_train=splitSentence(train)
#filling a dict of Wordtagzip
dic_trivial = {}

for i in content :
    if i[0] != '':
        #Wordtagzip
        dic_trivial[(i[0],i[1])] = (dic_trivial[(i[0],i[1])]+1) if (i[0],i[1]) in dic_trivial else 1


In [62]:
observations = ("midi",
                "en" ,
                "temps" ,
                "universel" ,
                "quatorze" ,
                "heures" ,
                "à" ,
                "Paris" ,
                "l'" ,
                "information" ,
                "continue" ,
                "sur" ,
                "RFI")
""" real :
midi O
en O
temps O
universel O
quatorze B-time
heures I-time
à O
Paris B-loc
l' O
information O
continue O
sur O
RFI B-org
"""
countWord = []
for word in observations:
    for wordTrv in dic_trivial.keys():
        if wordTrv[0] == word:
            countWord.append([dic_trivial[(wordTrv[0],wordTrv[1])],wordTrv[1]])

    m = max(countWord)
    print("Pour le mot :" + word + ", l'étiquette est :", m[1])
    countWord = []

Pour le mot :midi, l'étiquette est : O
Pour le mot :en, l'étiquette est : O
Pour le mot :temps, l'étiquette est : O
Pour le mot :universel, l'étiquette est : O
Pour le mot :quatorze, l'étiquette est : B-time
Pour le mot :heures, l'étiquette est : I-time
Pour le mot :à, l'étiquette est : O
Pour le mot :Paris, l'étiquette est : B-loc
Pour le mot :l', l'étiquette est : O
Pour le mot :information, l'étiquette est : O
Pour le mot :continue, l'étiquette est : O
Pour le mot :sur, l'étiquette est : O
Pour le mot :RFI, l'étiquette est : B-org


# TRAIN
For Viterbi Algorithm, we need the 3 parameters of the HMM:
λ(π,A,B) with
- π Initials probabilities ( Enter in the HMM knowing the i state)
- A, the transition matrix (Fullfilled with probabilities to go from state i to state j, i&j ∈ hmm states)
- B, distribution of probabilities associated to each i state

## π Initials probabilities

In [54]:
# List of firts taggers of each sentence
lst_firsts_taggers = []
lst_firsts_probs = []
delta=0.5#SMOOTHING
    
for sentence in spl_train:
    # firts taggers of each sentence
    lst_firsts_taggers.append(sentence[0][1])

# count of sentence in train
nb_sentence = len(lst_firsts_taggers) 

# Log(P) of each tagger
for e in aTagger:
    # Adding smoothing
    p = (delta+(lst_firsts_taggers.count(e)))/((delta*nb_sentence)+len(aTagger))
    p = math.log10(p) 
    lst_firsts_probs.append(p)

# final dic for PI
pi = dict(zip(aTagger,lst_firsts_probs))

print("Log(proba) : ")
for e,p in pi.items():
    print("\t", str(e) + ": "+ str(round(p,3)))

Log(proba) : 
	 O: 0.268
	 B-org: -1.536
	 I-org: -4.877
	 B-time: -1.743
	 I-time: -4.877
	 B-func: -2.305
	 I-func: -4.877
	 B-pers: -1.154
	 I-pers: -4.877
	 B-loc: -1.795
	 I-loc: -4.877
	 B-amount: -2.308
	 I-amount: -4.877
	 B-prod: -2.67
	 I-prod: -4.877


## A, the transition matrix

In [60]:
aDic = []
# for each tag ,dict
for tag in aTagger:
    probas = [0] * len(aTagger)
    dic = dict(zip(aTagger,probas))

    for sentence in spl_train:
        # in case the last tupl is empty
        nb= 2 if sentence[(len(sentence)-1)] == [''] else 1
        end = (len(sentence)-nb)
        #  On each sentence, number of Trasition between each tags
        for i in range(0, end):
            if sentence[i][1] == tag:
                dic[sentence[i+1][1]] += 1
    for d in dic:
        dic[d] += delta
    tot = sum(dic.values())
    for et in dic:
        dic[et] = math.log10((delta+dic[et])/(tot+len(taggersDiff)))

    aDic.append(dic)

# Final dict, transition matrix A
A = dict(zip(aTagger,aDic))

## B, distribution of probabilities associated to each i state

In [56]:
aDic = []
# dict for each tagger
for tag in aTagger:

    probas = [0] * len(aMot)
    dic = dict(zip(aMot,probas))
    
    for sentence in spl_train: 
        for tpl in sentence:
            if tpl != ['']:
                if tpl[1] == tag:
                    dic[tpl[0]] += 1
    for d in dic:
        dic[d] += 1
    tot = sum(dic.values())
    for et in dic:
        dic[et] = math.log10((delta+dic[et])/(tot+len(taggersDiff)))
    aDic.append(dic)    

emission = dict(zip(aTagger,aDic))

# VITERBI

In [57]:
#Now we have all our HMM parameters, Let s try it with Viterbi's Algorithm
#let's try it on dev dataset


file_dev = open("dev.ester1.cut.bio", "r")
dev = file_dev.read().split("\n\n")
devSplit = splitSentence(dev)
file_dev.close()

def error_sq(seq1,seq2 ):
    if(len(seq1)!=len(seq2)):
        return -1
    if(len(seq1)==0):
        return -1
    else:
        nb_true=0
        for i in range(0,len(seq1)):
            if(seq1[i]==seq2[i]):
                nb_true+=1
        return (1-(nb_true/len(seq1))) * 100

Viterbi is such simple, it's the product max between all Φt-1(i) *transition prob*  current tag prob
It works as Forward Algorithm, but, Forward use Sum, Viterbi use the product
We have 2 lists:
    -List of probabilities(j=max * i * transition * taggerconsumption), with j= current proba,i the previous layer one
    -List for the path wich gave us the max proba, We fill it when BACKTRACKING

In [58]:
def viterbi( unePhrase ):
    p_ath = []
    aObservations = []
    aTaggers = []
    aViterbiProba = []
    aViterbiTags = []
    lastDic = {}
    for word in unePhrase:
        aObservations.append( word[0] )
        aTaggers.append( word[1] )
    #We search for the list of taggers which have the best prob for our Observaation
    for obs in aObservations:
        dicProba = {}
        dicEti = {}

        # we take the value in PI on the tagger(i) if we just enter the graph
        if aObservations.index(obs) == 0:
            for et in aTagger:
                dicProba[et] = pi[et] + emission[et][obs]
                dicEti[et] = "-"            
        else:
            #else we do the product max between all Φt-1(i) transition with the current tag
            for et in aTagger:
                    probTemp = {}
                    try:
                        for tag in aTagger:
                            probTemp[tag] = ( lastDic[tag] + A[tag][et] + emission[et][obs] )
                        dicProba[et] = max(zip(probTemp.values(), probTemp.keys()))[0]
                        dicEti[et] = max(zip(probTemp.values(), probTemp.keys()))[1]
                    except:
                        pass

        aViterbiProba.append(dicProba)
        aViterbiTags.append(dicEti)
        lastDic = dicProba
    
    # BACKTRACKING TO GET THE BEST PATH, the one which gave us the max proba to get the observation
    i = len(aViterbiProba) - 1
    tag = ""
    while i >= 0:
        if i == len(aViterbiProba) - 1:
            dic = aViterbiProba[i]
            tag = max(zip(dic.values(), dic.keys()))[1]
            p_ath.append(tag)
        else:
            p_ath.append(aViterbiTags[i+1][tag])
            
        i -= 1

    #reversing the path       
    p_ath = p_ath[::-1]
    
    print( "\nChemin prédis:\t", p_ath, "\nChemin réel:\t", aTaggers )
    
    
    return round(error_sq( p_ath, aTaggers ), 2)

In [59]:
for sentence in devSplit:
    print("La sentence : ")
    sent = ''
    for word in sentence :
        sent += word[0] + " " 
    print(sent)
    print( viterbi( sentence ), "% d'erreur\n\n" )

La sentence : 
midi en temps universel quatorze heures à Paris l' information continue sur RFI 

Chemin prédis:	 ['O', 'O', 'O', 'O', 'O', 'I-time', 'O', 'B-loc', 'O', 'O', 'O', 'O', 'B-org'] 
Chemin réel:	 ['B-time', 'O', 'O', 'O', 'B-time', 'I-time', 'O', 'B-loc', 'O', 'O', 'O', 'O', 'B-org']
15.38 % d'erreur


La sentence : 
c' est avec Philippe Lecaplain bonjour 

Chemin prédis:	 ['O', 'O', 'O', 'B-pers', 'I-pers', 'O'] 
Chemin réel:	 ['O', 'O', 'O', 'B-pers', 'I-pers', 'O']
0.0 % d'erreur


La sentence : 
bonjour à tous 

Chemin prédis:	 ['O', 'O', 'O'] 
Chemin réel:	 ['O', 'O', 'O']
0.0 % d'erreur


La sentence : 
à la une de l' actualité 

Chemin prédis:	 ['O', 'O', 'O', 'O', 'O', 'O'] 
Chemin réel:	 ['O', 'O', 'O', 'O', 'O', 'O']
0.0 % d'erreur


La sentence : 
l' épidémie de pneumonie atypique 

Chemin prédis:	 ['O', 'O', 'O', 'O', 'O'] 
Chemin réel:	 ['O', 'O', 'O', 'O', 'O']
0.0 % d'erreur


La sentence : 
au cours de ces dernières heures huit personnes ont succombé à la mal

ValueError: max() arg is an empty sequence